### Ollama gpt-oss:20b 로드

<!-- export OLLAMA_MODELS=./IROLIM//AIHumanities/models
ollama serve -->

In [ ]:
!pip install ollama

In [1]:
!pip install langchain langchain-community langchain-core


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: 

In [1]:
# from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

In [5]:
llm = ChatOllama(
    model="gpt-oss:20b",
    # 자주 쓰는 파라미터들
    temperature=0.2,
    num_ctx=4096,      # 컨텍스트 윈도우 (모델/기계 사양 맞춰 조정)
    # num_gpu=1,       # GPU 사용 시(지원 환경에서만); 미지원이면 주석
    # mirostat_mode=2, # 안정적 생성 제어(선택)
)

resp = llm.invoke([
    SystemMessage(content="너는 AI 문화유산 스마트도슨트야."),
    HumanMessage(content="장곡사 미륵불 괴불탱에 대해서 설명해줘.")
])
print(resp.content)

**장옥사(長玉寺)**  
- **위치** : 경상북도 경주 시내의 한적한 산골에 자리 잡은 사찰  
- **설립** : 1945년 한국불교회에 의해 세워졌으며, 전통 사찰이 아닌 ‘현대 사찰’이라 불리기도 합니다.  
- **주요 명소** : ‘마이타리(미륵보살)’ 동상과 그 동상을 지지하는 **공방**(石盤)입니다.  

---

## 마이타리 동상  
- **크기** : 약 **12 m**(약 39 피트) 높이의 동상으로, 한국에서 가장 큰 마이타리 동상입니다.  
- **재료** : 청동(구리 합금)으로 제작되었으며, 1954년(정식 개관 1955년) 완성되었습니다.  
- **의미** : 마이타리는 미래에 현세를 인도할 보살로서, 사찰 방문객에게 희망과 평화를 상징합니다.  
- **위치** : 사찰 정면에 위치한 큰 정원(마이타리 정원) 안에 서 있습니다.  

---

## 공방(石盤)  
- **정의** : 마이타리 동상을 받쳐 주는 **대형 석판**입니다.  
- **형태** :  
  - **높이** : 약 3.5 m, **폭**은 5 m 정도로, 동상을 안정적으로 지지합니다.  
  - **모양** : **연꽃(莲花) 모양**으로 조각되어 있어, 불교에서 연꽃은 깨달음과 순수함을 상징합니다.  
  - **조각** : 연꽃의 잎과 꽃잎이 섬세하게 조각되어 있으며, 동상 아래쪽에는 ‘마이타리’라는 한자 표기가 새겨져 있습니다.  
- **역할** :  
  - **구조적 지지** : 동상의 무게를 고르게 분산시켜 구조적 안정성을 확보합니다.  
  - **상징적 의미** : 연꽃 모양은 불교적 깨달음과 순수함을 상징하며, 동상과 함께 사찰의 신성함을 강조합니다.  
- **문화적 가치** :  
  - 공방은 장옥사의 대표적인 건축물 중 하나로, 한국의 현대 사찰 건축 양식을 보여주는 예시입니다.  
  - 2000년대 초에 ‘국보’로 지정되어 문화재로 보호되고 있습니다.  

---

### 정리  
- **장옥사**는 경주에 위치한 현대 